In [21]:
import sys
sys.path.insert(1, "../pattern-explorer/src/")

In [22]:
from irf import irf_utils
from irf.ensemble import RandomForestClassifierWithWeights
import pandas
import pattern.preprocessing as prep
from pathlib import Path

In [23]:
data_folder = Path("../data/240109-a549-lps-data/")
lps = data_folder / "lps"
no_lps = data_folder / "no-lps"
lps, no_lps

(PosixPath('../data/240109-a549-lps-data/lps'),
 PosixPath('../data/240109-a549-lps-data/no-lps'))

In [24]:
len(list(lps.glob("**/*csv"))), len(list(no_lps.glob("**/*csv")))

(203, 191)

In [25]:
lps_data = [prep.spectra_pipeline(lps_file) for lps_file in lps.glob("**/*csv")]

In [27]:
lps_data[0].head()

inttime,"""IntTimeCal"" : 1.0","""driver"" : ""001.000.013.000_008.000.013.000_9.14.0.0"""
wavelength,,
2502.0,NaN,NaN
553.0,NaN,NaN
553.0,NaN,NaN
535.0,NaN,NaN
538.0,NaN,NaN
